In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('final_df2.csv',index_col=0)

In [3]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [4]:
from sklearn.preprocessing import StandardScaler

std_df = StandardScaler().fit_transform(X)
std_df = pd.DataFrame(std_df, index=X.index, columns=X.columns)
df2 = std_df
df2.head()

,ELMO2,CREB3L1,RPS11,PNMA1,MMP2,C10orf90,ZHX3,ERCC5,GPR98,RXFP3,...,SLC2A11,GRIP2,GPLD1,RAB8A,RXFP2,PIK3IP1,SLC39A6,SNRPD2,AQP7,CTSC
0,0.886827,0.349044,0.159261,-1.586876,-0.783517,-0.651792,-0.192552,-0.688943,1.790315,0.606942,...,0.618291,-0.244858,0.087421,-0.441970,0.805609,0.084007,-0.853409,-0.767943,-0.097567,0.208945
1,-0.076357,0.902533,0.026440,1.624105,1.396133,0.195322,0.394281,0.733565,1.460883,-0.288978,...,0.083736,-0.957388,0.095699,0.972672,0.691112,1.004964,-0.178898,-0.945141,0.843130,-1.062326
2,1.160544,0.875732,4.027129,0.072768,0.488563,-0.707819,-1.258655,2.353938,1.514641,-0.194159,...,0.561794,-0.680456,-0.348212,0.766775,-0.167253,0.878351,1.027238,-0.487267,0.630930,-0.273803
3,-0.093755,0.023215,-1.310286,0.819120,-0.815949,-0.407544,0.793497,-0.158602,-0.161729,1.060372,...,2.103530,-1.246807,-1.676863,0.213831,-1.585708,0.933448,0.740015,-1.050423,-0.987469,-0.644158
4,-0.249586,-0.633865,-0.325074,0.155425,0.019339,-0.462644,-0.576603,-1.569976,-1.943742,0.022223,...,-1.478282,0.569776,-0.794047,-0.079013,0.559841,-0.793164,-1.567624,0.733505,-0.957059,0.428773


In [5]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler
X_train, y_train = RandomUnderSampler(random_state=2021).fit_resample(X, y)

In [6]:
print("undersampling 적용 전 학습용 피처/레이블 데이터 세트 : ", X.shape, y.shape)
print('undersampling 적용 후 학습용 피처/레이블 데이터 세트 :', X_train.shape, y_train.shape)
print('undersampling 적용 후 값의 분포 :\n',pd.Series(y_train).value_counts() )

undersampling 적용 전 학습용 피처/레이블 데이터 세트 :  (590, 17814) (590,)
undersampling 적용 후 학습용 피처/레이블 데이터 세트 : (122, 17814) (122,)
undersampling 적용 후 값의 분포 :
 0    61
1    61
Name: label, dtype: int64


In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=109)
pca_array = pca.fit_transform(X_train)


In [8]:
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
# 주성분의 개수를 지정하지 않고 찾아줌
# 분산의 비율을 누적해서 더한 값이 95% 이상인 주성분의 개수
# 즉, 분산의 95%이상 유지
d = np.argmax(cumsum >= 0.99) + 1

In [9]:
d

109

In [12]:
dec=PCA(0.99)
x2=dec.fit_transform(X_train)
x2.shape

(122, 109)

In [14]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test=train_test_split(pca_array,y_train,test_size= 0.2, random_state=42)

In [15]:
def estimators_repeater(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC()],tr_slicer=(None,None),tst_slicer=(None,None),loops=300,scorer=accuracy_score,x=x,y=y):
    training_score={}
    testing_score={}
    timing={}
    for cl in estimators:
        cl_name=cl.__class__.__name__
        training_score[cl_name]=[]
        testing_score[cl_name]=[]
        timing[cl_name]=[]
    for i in range(loops):
        k1=time()
        x_train, x_test, y_train, y_test=train_test_split(x,y,random_state=i)
        for cl in estimators:
            a=time()
            cl_name=cl.__class__.__name__
            clean_cl=clone(cl)
            clean_cl.fit(x_train[tr_slicer[0]:tr_slicer[1]],y_train[tr_slicer[0]:tr_slicer[1]])
            training_score[cl_name].append(scorer(y_train[tr_slicer[0]:tr_slicer[1]],clean_cl.predict(x_train[tr_slicer[0]:tr_slicer[1]])))
            testing_score[cl_name].append(scorer(y_test[tst_slicer[0]:tst_slicer[1]],clean_cl.predict(x_test[tst_slicer[0]:tst_slicer[1]])))
            b=time()                                
            timing[cl_name].append(b-a)
        k2=time()
        print(f'number {i} out of {loops} took {k2-k1} seconds')
    global training_score_df
    training_score_df=pd.DataFrame(training_score)
    global testing_score_df
    testing_score_df=pd.DataFrame(testing_score)
    global timing_df
    timing_df=pd.DataFrame(timing)

In [16]:
estimators_repeater(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC(),LogisticRegression()],x=x,y=y)

number 0 out of 300 took 0.2902224063873291 seconds
number 1 out of 300 took 0.2862358093261719 seconds
number 2 out of 300 took 0.29421257972717285 seconds
number 3 out of 300 took 0.1765291690826416 seconds
number 4 out of 300 took 0.2822456359863281 seconds
number 5 out of 300 took 0.2812466621398926 seconds
number 6 out of 300 took 0.28623509407043457 seconds
number 7 out of 300 took 0.16655468940734863 seconds
number 8 out of 300 took 0.2812483310699463 seconds
number 9 out of 300 took 0.27725863456726074 seconds
number 10 out of 300 took 0.2752649784088135 seconds
number 11 out of 300 took 0.27725815773010254 seconds
number 12 out of 300 took 0.2782564163208008 seconds
number 13 out of 300 took 0.2812490463256836 seconds
number 14 out of 300 took 0.2712740898132324 seconds
number 15 out of 300 took 0.28025197982788086 seconds
number 16 out of 300 took 0.2782554626464844 seconds
number 17 out of 300 took 0.2782559394836426 seconds
number 18 out of 300 took 0.2782552242279053 secon

number 154 out of 300 took 0.2792534828186035 seconds
number 155 out of 300 took 0.2792530059814453 seconds
number 156 out of 300 took 0.2732686996459961 seconds
number 157 out of 300 took 0.2792544364929199 seconds
number 158 out of 300 took 0.16954612731933594 seconds
number 159 out of 300 took 0.27426624298095703 seconds
number 160 out of 300 took 0.2762613296508789 seconds
number 161 out of 300 took 0.27227187156677246 seconds
number 162 out of 300 took 0.2762627601623535 seconds
number 163 out of 300 took 0.1685490608215332 seconds
number 164 out of 300 took 0.17253804206848145 seconds
number 165 out of 300 took 0.2762618064880371 seconds
number 166 out of 300 took 0.27426767349243164 seconds
number 167 out of 300 took 0.282245397567749 seconds
number 168 out of 300 took 0.2792534828186035 seconds
number 169 out of 300 took 0.2792518138885498 seconds
number 170 out of 300 took 0.29122185707092285 seconds
number 171 out of 300 took 0.2812492847442627 seconds
number 172 out of 300 t

In [17]:
training_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.0,300.0,300.0,300.0
mean,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0


In [18]:
testing_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.000000,300.000000,300.000000,300.000000
mean,0.966267,0.973867,0.989467,0.995333
std,0.032023,0.029256,0.019381,0.012862
min,0.880000,0.840000,0.840000,0.960000
25%,0.960000,0.960000,0.960000,1.000000
50%,0.960000,0.960000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [19]:
timing_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.000000,300.000000,300.000000,300.000000
mean,0.155531,0.083557,0.002902,0.008344
std,0.002691,0.046734,0.000402,0.000930
min,0.150596,0.002991,0.001993,0.004986
25%,0.153590,0.003991,0.002991,0.007978
50%,0.154587,0.109707,0.002992,0.007979
75%,0.156582,0.111701,0.002992,0.008976
max,0.174534,0.124666,0.003991,0.012965


In [26]:
def estimators_repeater2(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC()],tr_slicer=(None,None),tst_slicer=(None,None),loops=300,scorer=recall_score,x=x,y=y):
    training_score={}
    testing_score={}
    timing={}
    for cl in estimators:
        cl_name=cl.__class__.__name__
        training_score[cl_name]=[]
        testing_score[cl_name]=[]
        timing[cl_name]=[]
    for i in range(loops):
        k1=time()
        x_train, x_test, y_train, y_test=train_test_split(x,y,random_state=i)
        for cl in estimators:
            a=time()
            cl_name=cl.__class__.__name__
            clean_cl=clone(cl)
            clean_cl.fit(x_train[tr_slicer[0]:tr_slicer[1]],y_train[tr_slicer[0]:tr_slicer[1]])
            training_score[cl_name].append(scorer(y_train[tr_slicer[0]:tr_slicer[1]],clean_cl.predict(x_train[tr_slicer[0]:tr_slicer[1]])))
            testing_score[cl_name].append(scorer(y_test[tst_slicer[0]:tst_slicer[1]],clean_cl.predict(x_test[tst_slicer[0]:tst_slicer[1]])))
            b=time()                                
            timing[cl_name].append(b-a)
        k2=time()
        print(f'number {i} out of {loops} took {k2-k1} seconds')
    global training_score_df
    training_score_df=pd.DataFrame(training_score)
    global testing_score_df
    testing_score_df=pd.DataFrame(testing_score)
    global timing_df
    timing_df=pd.DataFrame(timing)

In [27]:
estimators_repeater2(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC(),LogisticRegression()],x=x,y=y)

number 0 out of 300 took 0.28623223304748535 seconds
number 1 out of 300 took 0.28424072265625 seconds
number 2 out of 300 took 0.2822446823120117 seconds
number 3 out of 300 took 0.204453706741333 seconds
number 4 out of 300 took 0.3031885623931885 seconds
number 5 out of 300 took 0.2892270088195801 seconds
number 6 out of 300 took 0.29720544815063477 seconds
number 7 out of 300 took 0.18650364875793457 seconds
number 8 out of 300 took 0.32114076614379883 seconds
number 9 out of 300 took 0.29620838165283203 seconds
number 10 out of 300 took 0.2932164669036865 seconds
number 11 out of 300 took 0.290224552154541 seconds
number 12 out of 300 took 0.28822922706604004 seconds
number 13 out of 300 took 0.28523731231689453 seconds
number 14 out of 300 took 0.2812495231628418 seconds
number 15 out of 300 took 0.28523707389831543 seconds
number 16 out of 300 took 0.2982039451599121 seconds
number 17 out of 300 took 0.3400912284851074 seconds
number 18 out of 300 took 0.28922557830810547 second

number 154 out of 300 took 0.3001973628997803 seconds
number 155 out of 300 took 0.32213878631591797 seconds
number 156 out of 300 took 0.3121659755706787 seconds
number 157 out of 300 took 0.2952108383178711 seconds
number 158 out of 300 took 0.18051695823669434 seconds
number 159 out of 300 took 0.34108877182006836 seconds
number 160 out of 300 took 0.354053258895874 seconds
number 161 out of 300 took 0.3091731071472168 seconds
number 162 out of 300 took 0.3061816692352295 seconds
number 163 out of 300 took 0.18450665473937988 seconds
number 164 out of 300 took 0.18151569366455078 seconds
number 165 out of 300 took 0.325131893157959 seconds
number 166 out of 300 took 0.3061819076538086 seconds
number 167 out of 300 took 0.3201427459716797 seconds
number 168 out of 300 took 0.31017065048217773 seconds
number 169 out of 300 took 0.3001973628997803 seconds
number 170 out of 300 took 0.2872323989868164 seconds
number 171 out of 300 took 0.29321718215942383 seconds
number 172 out of 300 t

In [28]:
testing_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.000000,300.000000,300.0,300.0
mean,0.963810,0.971070,1.0,1.0
std,0.045642,0.047851,0.0,0.0
min,0.818182,0.733333,1.0,1.0
25%,0.928571,0.933333,1.0,1.0
50%,1.000000,1.000000,1.0,1.0
75%,1.000000,1.000000,1.0,1.0
max,1.000000,1.000000,1.0,1.0


In [46]:
from sklearn.decomposition import PCA

pca = PCA(n_components=590)
pca_array = pca.fit_transform(df2)
pca_df = pd.DataFrame(pca_array, index=df2.index,
                      columns=[f"pca{num+1}" for num in range(590)])
pca_df.head()

,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,...,pca581,pca582,pca583,pca584,pca585,pca586,pca587,pca588,pca589,pca590
0,-33.804476,-2.787982,-5.770776,-30.958220,5.887185,13.407046,2.691613,-14.574579,-9.552975,35.365711,...,0.196188,0.154316,0.327665,0.057420,0.454254,0.043743,-0.494110,0.163552,0.313049,2.829963e-15
1,12.129357,-20.493103,13.751279,-29.977843,-8.738578,-3.651478,-16.750653,18.264358,0.222539,-23.017786,...,-1.231051,0.436648,0.278043,0.841787,0.056323,0.110008,0.127392,0.172217,-0.086814,2.829963e-15
2,-4.647118,-27.232655,-10.584373,-4.731128,-26.303871,-22.174773,5.284348,1.192801,23.295138,2.967032,...,-0.016966,0.286153,0.738451,-0.027324,0.026627,-0.631649,0.330128,-0.128202,0.091551,2.829963e-15
3,-7.638704,-57.561760,7.498658,5.308718,0.283737,-2.635175,-8.665971,1.103797,-17.317805,-5.960301,...,0.261537,-0.382600,0.365680,0.041219,0.090431,-0.025359,0.661129,0.442092,-0.349987,2.829963e-15
4,-31.522639,76.837420,-26.187440,-6.820377,12.824112,-1.214566,11.631132,-9.568467,-10.434659,-4.633602,...,-0.150812,0.078911,0.046604,-0.195311,0.079283,-0.053218,0.263967,0.386297,-0.499102,2.829963e-15


In [49]:
result = pd.DataFrame({'설명가능한 분산 비율(고윳값)':pca.explained_variance_,
             '기여율':pca.explained_variance_ratio_},
            index=np.array([f"pca{num+1}" for num in range(590)]))
result['누적기여율'] = result['기여율'].cumsum()
result

,설명가능한 분산 비율(고윳값),기여율,누적기여율
pca1,1.553460e+03,8.705178e-02,0.087052
pca2,1.347472e+03,7.550873e-02,0.162561
pca3,7.642334e+02,4.282560e-02,0.205386
pca4,6.858864e+02,3.843524e-02,0.243821
pca5,4.964501e+02,2.781974e-02,0.271641
...,...,...,...
pca586,2.475582e+00,1.387250e-04,0.999615
pca587,2.370171e+00,1.328181e-04,0.999748
pca588,2.318629e+00,1.299298e-04,0.999878
pca589,2.185088e+00,1.224465e-04,1.000000


In [51]:
# MNIST 데이터 사용하기
from sklearn.datasets import fetch_openml

# 784개의 픽셀(특성)을 가지고 있는 데이터 셋
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)